In [5]:
import torch
import data as Data
import model as Model
import argparse
import logging
import core.logger as Logger
import core.metrics as Metrics
from core.wandb_logger import WandbLogger
from tensorboardX import SummaryWriter
import os
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim
import gc
import random
from torch import optim

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

def calculate_ssim(sr_img, hr_img):
    if sr_img.ndim == 3:
        sr_img = cv2.cvtColor(sr_img, cv2.COLOR_BGR2GRAY)
    if hr_img.ndim == 3:
        hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2GRAY)

    sr_img = sr_img.astype(float)
    hr_img = hr_img.astype(float)

    ssim_value, _ = ssim(sr_img, hr_img, full=True)

    return ssim_value


class EmaLRScheduler:
    def __init__(self, optimizer, ema_decay):
        self.ema_decay = ema_decay

    def get_shadow_lr(self):
        return self.ema_decay
    

def create_directories(opt):
    os.makedirs(opt['path']['log'], exist_ok=True)
    os.makedirs(opt['path']['results'], exist_ok=True)
    os.makedirs(opt['path']['tb_logger'], exist_ok=True)


def objective(trial):
    print("Objective function called.")
    print(os.getcwd())
    torch.cuda.empty_cache()
    gc.collect()

    # Modify the optuna_config with the sampled hyperparameters from the trial
    # optuna_config["model"]["unet"]["inner_channel"] = trial.suggest_int("inner_channel", 32, 128)
    #optuna_config["model"]["unet"]["channel_multiplier"] = trial.suggest_categorical("channel_multiplier", [[1, 2, 4],[1, 2, 4, 8, 16]])
    #optuna_config["model"]["unet"]["res_blocks"] = trial.suggest_int("norm_groups", 16, 32, step = 16)
    #optuna_config["model"]["unet"]["attn_res"] = trial.suggest_categorical("attn_res", [[16], [32]])
    #optuna_config["model"]["unet"]["res_blocks"] = trial.suggest_int("res_blocks", 2, 6, step = 2)
    #optuna_config["model"]["unet"]["dropout"] = trial.suggest_float("dropout", 0.15, 0.5)

    optuna_config["model"]["beta_schedule"]["train"]["schedule"] = trial.suggest_categorical(['quad', 'linear', 'warmup10', 'warmup50', 'const', 'cosine'])
    optuna_config["model"]["beta_schedule"]["val"]["schedule"] = optuna_config["model"]["beta_schedule"]["train"]["schedule"]

    #optuna_config["model"]["beta_schedule"]["train"]["linear_start"] = trial.suggest_loguniform("linear_start_train", 1e-8, 1e-5)
    #optuna_config["model"]["beta_schedule"]["val"]["linear_start"] = optuna_config["model"]["beta_schedule"]["train"]["linear_start"] #trial.suggest_loguniform("linear_start_val", 1e-9, 1e-6)

    #optuna_config["model"]["beta_schedule"]["train"]["linear_end"] = trial.suggest_loguniform("linear_end_train", 5e-3, 5e-1)
    #optuna_config["model"]["beta_schedule"]["val"]["linear_end"] = optuna_config["model"]["beta_schedule"]["train"]["linear_end"] #trial.suggest_loguniform("linear_end_val", 1e-4, 1e-1)

    #optuna_config["train"]["optimizer"]["lr"] = trial.suggest_loguniform("lr", 5e-5, 5e-3)
    #optuna_config["train"]["optimizer"]["type"] = trial.suggest_categorical("type_optimizer", ["adam"]) # ['SGD']

    #optuna_config["train"]["optimizer"]["finetune_norm"] = trial.suggest_categorical("finetune_norm", [False, True])
    #optuna_config["train"]["optimizer"]["which_model_G"] = trial.suggest_categorical("which_model_G", ["sr3"])

    optuna_config["path"]["log"] = f"experiments/logs/trial_{trial.number}"
    optuna_config["path"]["results"] = f"experiments/results/trial_{trial.number}"
    optuna_config["path"]["tb_logger"] = f"experiments/tb_logs/trial_{trial.number}"

    opt = optuna_config

    create_directories(opt)

    # logging
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True

    logger = Logger.setup_logger(None, opt['path']['log'], 'train', level=logging.INFO, screen=True)
    logger_val = logging.getLogger('val')  # Retrieve the logger when needed

    logger = logging.getLogger('base')
    logger.info(Logger.dict2str(opt))

    tb_logger = SummaryWriter(log_dir=opt['path']['tb_logger'])

    logger.info("Trial {}: Parameters:".format(trial.number))

    '''
    logger.info("attn_res: {}".format(optuna_config["model"]["unet"]["attn_res"]))
    logger.info("res_blocks: {}".format(optuna_config["model"]["unet"]["res_blocks"]))
    logger.info("dropout: {}".format(optuna_config["model"]["unet"]["dropout"]))
    logger.info("n_timestep_train: {}".format(optuna_config["model"]["beta_schedule"]["train"]["n_timestep"]))
    logger.info("n_timestep_val: {}".format(optuna_config["model"]["beta_schedule"]["val"]["n_timestep"]))
    logger.info("linear_start_train: {}".format(optuna_config["model"]["beta_schedule"]["train"]["linear_start"]))
    logger.info("linear_start_val: {}".format(optuna_config["model"]["beta_schedule"]["val"]["linear_start"]))
    logger.info("linear_end_train: {}".format(optuna_config["model"]["beta_schedule"]["train"]["linear_end"]))
    logger.info("linear_end_val: {}".format(optuna_config["model"]["beta_schedule"]["val"]["linear_end"]))
    logger.info("lr: {}".format(optuna_config["train"]["optimizer"]["lr"]))
    logger.info("type_optimizer: {}".format(optuna_config["train"]["optimizer"]["type"]))
    logger.info("finetune_norm: {}".format(optuna_config["train"]["optimizer"]["finetune_norm"]))
    logger.info("which_model_G: {}".format(optuna_config["train"]["optimizer"]["which_model_G"]))
    '''

    wandb_logger = None

    # dataset
    for phase, dataset_opt in opt['datasets'].items():
        if phase == 'train' and opt['phase'] != 'val':
            train_set = Data.create_dataset(dataset_opt, phase)
            train_loader = Data.create_dataloader(
                train_set, dataset_opt, phase)
        elif phase == 'val':
            val_set = Data.create_dataset(dataset_opt, phase)
            val_loader = Data.create_dataloader(
                val_set, dataset_opt, phase)
    logger.info('Initial Dataset Finished')

    # model
    diffusion = Model.create_model(opt)
    logger.info('Initial Model Finished')

    # Train
    current_step = diffusion.begin_step
    current_epoch = diffusion.begin_epoch
    n_iter = opt['train']['n_iter']

    if opt['path']['resume_state']:
        logger.info('Resuming training from epoch: {}, iter: {}.'.format(
            current_epoch, current_step))

    diffusion.set_new_noise_schedule(
        opt['model']['beta_schedule'][opt['phase']], schedule_phase=opt['phase'])
    if opt['phase'] == 'train':
        while current_step < n_iter:
            current_epoch += 1
            for _, train_data in enumerate(train_loader):
                current_step += 1
                if current_step > n_iter:
                    break
                diffusion.feed_data(train_data)
                diffusion.optimize_parameters()
                # log
                if current_step >= opt['train']['ema_scheduler']['step_start_ema'] and current_step % opt['train']['ema_scheduler']['update_ema_every'] == 0:
                    diffusion.update_lr(opt['train']['ema_scheduler']['ema_decay'])


                if current_step % opt['train']['print_freq'] == 0:
                    logs = diffusion.get_current_log()
                    message = '<epoch:{:3d}, iter:{:8,d}> '.format(
                        current_epoch, current_step)
                    for k, v in logs.items():
                        message += '{:s}: {:.4e} '.format(k, v)
                        tb_logger.add_scalar(k, v, current_step)
                    logger.info(message)

                    if wandb_logger:
                        wandb_logger.log_metrics(logs)

                # validation
                if current_step % opt['train']['val_freq'] == 0:
                    avg_psnr = 0.0
                    avg_ssim = 0.0
                    idx = 0
                    result_path = '{}/{}'.format(opt['path']
                                                ['results'], current_epoch)
                    os.makedirs(result_path, exist_ok=True)

                    diffusion.set_new_noise_schedule(
                        opt['model']['beta_schedule']['val'], schedule_phase='val')
                    for _,  val_data in enumerate(val_loader):
                        idx += 1
                        print('feeding data')
                        diffusion.feed_data(val_data)
                        print('testing diffusion')
                        diffusion.test(continous=False)
                        visuals = diffusion.get_current_visuals()
                        sr_img = Metrics.tensor2img(visuals['SR'])  # uint8
                        hr_img = Metrics.tensor2img(visuals['HR'])  # uint8
                        lr_img = Metrics.tensor2img(visuals['LR'])  # uint8
                        fake_img = Metrics.tensor2img(visuals['INF'])  # uint8

                        # generation
                        Metrics.save_img(
                            hr_img, '{}/{}_{}_hr.png'.format(result_path, current_step, idx))
                        Metrics.save_img(
                            sr_img, '{}/{}_{}_sr.png'.format(result_path, current_step, idx))
                        Metrics.save_img(
                            lr_img, '{}/{}_{}_lr.png'.format(result_path, current_step, idx))
                        Metrics.save_img(
                            fake_img, '{}/{}_{}_inf.png'.format(result_path, current_step, idx))
                        
                        def ensure_3d(img):
                            return np.expand_dims(img, axis=2) if len(img.shape)==2 else img

                        fake_img = ensure_3d(fake_img)
                        sr_img = ensure_3d(sr_img)
                        hr_img = ensure_3d(hr_img)

                        tb_logger.add_image(
                            'Iter_{}'.format(current_step),
                            np.transpose(np.concatenate(
                                (fake_img, sr_img, hr_img), axis=1), [2, 0, 1]),
                            idx)
                        avg_psnr += Metrics.calculate_psnr(
                            sr_img, hr_img)
                        avg_ssim += Metrics.calculate_ssim(
                            sr_img, hr_img)

                        if wandb_logger:
                            wandb_logger.log_image(
                                f'validation_{idx}', 
                                np.concatenate((fake_img, sr_img, hr_img), axis=1)
                            )  

                    avg_psnr = avg_psnr / idx
                    avg_ssim = avg_ssim / idx
                    diffusion.set_new_noise_schedule(
                        opt['model']['beta_schedule']['train'], schedule_phase='train')
                    # log
                    logger.info('# Validation # PSNR: {:.2}'.format(avg_psnr))
                    logger.info('# Validation # SSIM: {:.2}'.format(avg_ssim))
                    logger_val = logging.getLogger('val')  # validation logger
                    logger_val.info('<epoch:{:3d}, iter:{:8,d}> psnr: {:.4e}'.format(
                        current_epoch, current_step, avg_psnr))
                    # tensorboard logger
                    tb_logger.add_scalar('psnr', avg_psnr, current_step)
                    tb_logger.add_scalar('ssim', avg_ssim, current_step)

                    if wandb_logger:
                        wandb_logger.log_metrics({
                            'validation/val_psnr': avg_psnr,
                            'validation/val_step': val_step
                        })
                        val_step += 1

                if current_step % opt['train']['save_checkpoint_freq'] == 0:
                    logger.info('Saving models and training states.')
                    diffusion.save_network(current_epoch, current_step)

                    if wandb_logger and opt['log_wandb_ckpt']:
                        wandb_logger.log_checkpoint(current_epoch, current_step)

            if wandb_logger:
                wandb_logger.log_metrics({'epoch': current_epoch-1})

        # save model
        logger.info('End of training.')
    else:
        logger.info('Begin Model Evaluation.')
        avg_psnr = 0.0
        avg_ssim = 0.0
        idx = 0
        result_path = '{}/trial_{}/{}'.format(opt['path']['results'], trial.number, current_epoch)
        os.makedirs(result_path, exist_ok=True)
        for _,  val_data in enumerate(val_loader):
            idx += 1
            diffusion.feed_data(val_data)
            diffusion.test(continous=True)
            visuals = diffusion.get_current_visuals()
            #print(type(visuals))

            hr_img = Metrics.tensor2img(visuals['HR'])  # uint8
            lr_img = Metrics.tensor2img(visuals['LR'])  # uint8
            fake_img = Metrics.tensor2img(visuals['INF'])  # uint8

            sr_img_mode = 'grid'
            if sr_img_mode == 'single':
                #print('single')
                # single img series
                sr_img = visuals['SR']  # uint8
                sample_num = sr_img.shape[0]
                for iter in range(0, sample_num):
                    Metrics.save_img(
                        Metrics.tensor2img(sr_img[iter]), '{}/{}_{}_sr_{}.png'.format(result_path, current_step, idx, iter))
            else:
                # grid img
                sr_img = Metrics.tensor2img(visuals['SR'])  # uint8
                Metrics.save_img(
                    sr_img, '{}/{}_{}_sr_process.png'.format(result_path, current_step, idx))
                Metrics.save_img(
                    Metrics.tensor2img(visuals['SR'][-1]), '{}/{}_{}_sr.png'.format(result_path, current_step, idx))

            Metrics.save_img(
                hr_img, '{}/{}_{}_hr.png'.format(result_path, current_step, idx))
            Metrics.save_img(
                lr_img, '{}/{}_{}_lr.png'.format(result_path, current_step, idx))
            Metrics.save_img(
                fake_img, '{}/{}_{}_inf.png'.format(result_path, current_step, idx))

            # generation
            eval_psnr = Metrics.calculate_psnr(Metrics.tensor2img(visuals['SR'][-1]), hr_img)
            eval_ssim = calculate_ssim(Metrics.tensor2img(visuals['SR'][-1]), hr_img)
            #calculate_ssim(sr_img, hr_img)

            avg_psnr += eval_psnr
            avg_ssim += eval_ssim
            print(eval_ssim)

            if wandb_logger and opt['log_eval']:
                wandb_logger.log_eval_data(fake_img, Metrics.tensor2img(visuals['SR'][-1]), hr_img, eval_psnr, eval_ssim)

        avg_psnr = avg_psnr / idx
        avg_ssim = avg_ssim / idx
        

        logger.info('# Validation # PSNR: {:.4e}'.format(avg_psnr))
        logger.info('# Validation # SSIM: {:.4e}'.format(avg_ssim))
        logger_val = logging.getLogger('val')  # validation logger
        logger_val.info('<epoch:{:3d}, iter:{:8,d}> psnr: {:.4e}, ssim：{:.4e}'.format(
            current_epoch, current_step, avg_psnr, avg_ssim))
        
        tb_logger.add_scalar('psnr', avg_psnr, current_step)
        tb_logger.add_scalar('ssim', avg_ssim, current_step)
    
    return avg_ssim


import optuna
from optuna.integration import PyTorchLightningPruningCallback

study = optuna.create_study(direction="maximize")  # Change to 'minimize' if optimizing a loss
study.optimize(objective, n_trials=20)

# Get the best hyperparameters from the study
best_params = study.best_params
print("Best Hyperparameters:", best_params)

ModuleNotFoundError: No module named 'tensorboardX'

: 

!pip install SummaryWriter

# OPTUNA DICT

In [4]:
########### optuna imports
from optuna.integration import PyTorchLightningPruningCallback


optuna_config = {
    "name": "soilCT",
    "phase": "train",
    "gpu_ids": [1],
    "distributed": False,
    "path": {
        "log": "logs",
        "tb_logger": "tb_logs",
        "results": "results",
        "checkpoint": "experiments/checkpoint",
        "resume_state": None,
    },
    "datasets": {
        "train": {
            "name": "soilCT",
            "mode": "LRHR",
            "dataroot": "C:\\Users\\neuro-ws\\data_local\\toy_data\\train",
            "datatype": "img",
            "l_resolution": 32,
            "r_resolution": 128,
            "batch_size": 8,
            "num_workers": 4,
            "use_shuffle": True,
            "data_len": 1000,
        },
        "val": {
            "name": "soilVal",
            "mode": "LRHR",
            "dataroot": "C:\\Users\\neuro-ws\\data_local\\toy_data\\val",
            "datatype": "img",
            "l_resolution": 32,
            "r_resolution": 128,
            "data_len": 2,
        },
    },
    "model": {
        "which_model_G": "sr3",
        "finetune_norm": False,
        "unet": {
            "in_channel": 2,
            "out_channel": 1,
            "inner_channel": 64,
            "norm_groups": 32,
            "channel_multiplier": [1, 2, 4, 8],
            "attn_res": [32],
            "res_blocks": 4,
            "dropout": 0.2,
        },
        "beta_schedule": {
            "train": {
                "schedule": "linear",
                "n_timestep": 5000,
                "linear_start": 1e-8,
                "linear_end": 1e-2,
            },
            "val": {
                "schedule": "linear",
                "n_timestep": 5000,
                "linear_start": 1e-8,
                "linear_end": 1e-2,
            },
        },
        "diffusion": {"image_size": 256, "channels": 1, "conditional": True},
    },
    "train": {
        "n_iter": 12000,
        "val_freq": 4000,
        "save_checkpoint_freq": 12000,
        "print_freq": 1000,
        "optimizer": {"type": "adam", "lr": 1e-4},
        "ema_scheduler": {"step_start_ema": 5000, "update_ema_every": 1000, "ema_decay": 0.9},
    },
    "wandb": {"project": "sr_soilCT"},
}

In [3]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [5]:
!nvidia-smi

Tue Dec 19 12:23:58 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.92                 Driver Version: 545.92       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090      WDDM  | 00000000:01:00.0 Off |                  Off |
|  0%   43C    P8              19W / 450W |      0MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from PIL import Image, ImageFilter
import os

os.chdir('c:\\Users\\neuro-ws\\data_local\\comprs')

def calculate_pore_space(image, threshold):
    return np.round(np.sum(image < threshold) / image.size * 100, 2)

def non_local_mean_filter(image, vals):
    j, k, m = vals
    # Use OpenCV's fast non-local means denoising function
    denoised_image = cv2.fastNlMeansDenoising (image, None, j, k, m)
    denoised_image_pil = Image.fromarray(denoised_image)
    return denoised_image_pil

def downscale_and_filter(image, factor=2, interpolation=cv2.INTER_CUBIC, vals = (20, 7, 21)):
    image_np = np.array(image)
    
    upscaled_image = cv2.resize(image_np, dsize = None, fx=1/factor, fy=1/factor, interpolation=interpolation)

    filtered_image = non_local_mean_filter(upscaled_image, vals)

    return filtered_image

# Read the input image
input_image_path = "path/to/your/image.jpg"
input_image = cv2.imread(input_image_path)

threshold = 0.1

# Load images in grayscale mode
lr = cv2.imread('189_lr.png', cv2.IMREAD_GRAYSCALE) / 255.0
hr = Image.open('189_hr.png').convert("L")

# Get the size of LR image for resizing
lr_size = lr.shape[::-1]

# Define resampling methods
resample_methods = [Image.BICUBIC, Image.LANCZOS, (1,1,4), (2,1,6), (4,2,8), (8,2,10), (10,5,16)]
method_names = ['GAUSSIAN_B','GAUSSIAN_L', 'NON_LOCAL_MEAN', 'NON_LOCAL_MEAN2', 'NON_LOCAL_MEAN3', 'NON_LOCAL_MEAN4', 'NON_LOCAL_MEAN5']

# Start plot
fig, axs = plt.subplots(1, len(resample_methods)+1, figsize=(27,18))

# Show LR image first
axs[0].imshow(lr, cmap='gray')
axs[0].axis('off')
pore_space_orig = calculate_pore_space(np.array(lr), threshold)
axs[0].set_title(f'LR \nPore Space: {pore_space_orig:.2f}%')

# For each resampling method, resize HR, calculate metrics and show image

for (i), (method, name) in enumerate(zip(resample_methods, method_names), start=1):
    # Resize the image
    if 'GAUSSIAN' in name:
        pyramid = hr.copy()
        pyramid_images = [pyramid]
        for n in range(2): #(range(3) - x8, range(2) - x4, range(1) - x2) кратность увеличения = степени двойки range
            if n == 1 or name == 'GAUSSIAN_L':
                pyramid = pyramid.filter(ImageFilter.GaussianBlur(1.9))
            pyramid = pyramid.resize((pyramid.width // 2, pyramid.height // 2), method) #Image.LANCZOS #Image.NEAREST
            pyramid_images.append(pyramid)
        resized_image_np = np.array(pyramid_images[-1]) / 255.0
    elif 'NON_LOCAL_MEAN' in name:
        pyramid = hr.copy()
        pyramid_images = [pyramid]
        for n in range(2): #(range(3) - x8, range(2) - x4, range(1) - x2) кратность увеличения = степени двойки range
            if n == 1:
                pyramid = pyramid.filter(ImageFilter.GaussianBlur(1.8))
            pyramid = downscale_and_filter(pyramid, factor=2, vals=method)  
            pyramid_images.append(pyramid)
        resized_image_np = np.array(pyramid_images[-1]) / 255.0
    else:
        resized_image = hr.resize(lr_size, method)
        resized_image_np = np.array(resized_image) / 255.0

    # Calculate PSNR, SSIM, and pore space
    psnr_val = psnr(lr, resized_image_np)
    ssim_val = ssim(lr, resized_image_np)
    pore_space = calculate_pore_space(resized_image_np, threshold)

    # Display the image
    axs[i].imshow(resized_image_np, cmap='gray')
    axs[i].axis('off')
    axs[i].set_title(f'{name}\nPSNR: {psnr_val:.2f}\nSSIM: {ssim_val:.2f}\nPore Space: {pore_space:.2f}%')

plt.tight_layout()
plt.show()